In [ ]:
import s3fs, pyarrow.parquet as pq, pandas as pd
import json

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["AWS_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_ALLOW_HTTP"] = "true"
os.environ["AWS_S3_ALLOW_UNSAFE_RENAME"] = "true"

from deltalake import DeltaTable

dt = DeltaTable("s3://deltabucket/bronze/wholeCorp_delta_raw")
df = dt.to_pandas()


In [ ]:
prefix = "deltabucket/gold/wholeCorp_delta"
files = fs.glob(f"{prefix}/*.parquet")

dfs = []
for f in files:
    table = pq.ParquetDataset(f"s3://{f}", filesystem=fs).read()
    df = table.to_pandas()
    dfs.append(df)

df_whole_gold = pd.concat(dfs)

df_merged = df_whole_silver.merge(df_whole_gold[['統一編號','features']], on= "統一編號", how='left')
df_merged = df_merged[['統一編號', '公司名稱', '負責人', '登記地址', '資本額', '營業項目及代碼表', '縣市名稱', '區域名稱',
        '類別_全', '官網', '電話', 'features']]
df_merged.rename(columns={'features':'features_vector'}, inplace=True)
df_merged

In [ ]:
# --- Read Delta from MinIO ---
storage_options = {
    "AWS_ACCESS_KEY_ID": "minioadmin",
    "AWS_SECRET_ACCESS_KEY": "minioadmin",
    "AWS_ENDPOINT_URL": MINIO_ENDPOINT,
}

dt = DeltaTable(DELTA_PATH, storage_options=storage_options)

# Convert to Pandas (you can chunk if too big)
df = dt.to_pandas()

# --- Send to Elasticsearch ---
es = Elasticsearch(ES_URL)

def docs():
    for rec in df.to_dict(orient="records"):
        yield {"_index": ES_INDEX, "_source": rec}

helpers.bulk(es, docs())
print(f"Inserted {len(df)} records into {ES_INDEX}")


# Check log

In [ ]:
import s3fs, json

fs = s3fs.S3FileSystem(
    key="minioadmin",
    secret="minioadmin",
    client_kwargs={"endpoint_url": "http://localhost:9000"},
)

log_path = "deltabucket/gold/wholeCorp_clusters/_delta_log/00000000000000000000.json"

In [ ]:
with fs.open(log_path) as f:
    for line in f:
        print(line)
        obj = json.loads(line)

In [ ]:
with fs.open(log_path) as f:
    for line in f:
        if not line.strip():
            continue
        obj = json.loads(line)
        if "metaData" in obj:
            print("Schema string:")
            print(obj["metaData"]["schemaString"])

# Read parquet cluster gold

In [ ]:
prefix = "deltabucket/gold/wholeCorp_clusters"
files = fs.glob(f"{prefix}/*.parquet")

dfs = []
for f in files:
    table = pq.ParquetDataset(f"s3://{f}", filesystem=fs).read()
    df = table.to_pandas()
    dfs.append(df)

df_whole_gold_cluster = pd.concat(dfs)

# df_merged = df_whole_silver.merge(df_whole_gold[['統一編號','features']], on= "統一編號", how='left')
# df_merged = df_merged[['統一編號', '公司名稱', '負責人', '登記地址', '資本額', '營業項目及代碼表', '縣市名稱', '區域名稱',
#         '類別_全', '官網', '電話', 'features']]
# df_merged.rename(columns={'features':'features_vector'}, inplace=True)

In [ ]:
df_whole_gold_cluster